# BIOPS QS JSON Option Use Case 1: Move one dashboard to another account with assets as bundle API #
Author: Ying Wang, Sr. SA in Gen AI, QuickSight

Created Time: June 2025

## Update pip and boto3

In [ ]:
!pip install --upgrade pip
!pip install --upgrade boto3
get_ipython().system('pip install --upgrade ipynb')

## Only run it one time!!!

The cell below is to update the bucket policy to allow cross-accounts access: let the targe account to copy the assets in source account.
Please update the template path if you have a new assets bundle to be imported into the target account

In [ ]:
import boto3
import json
from botocore.exceptions import ClientError

# === Configuration ===
bucket_name = "biops-version-control-demo-2025"
source_account_id = ""  # Replace with the AWS Account ID to allow
target_account_id = ""
role_name = ""  # Replace with the role name you want to allow access

# === Initialize S3 client ===
s3 = boto3.client("s3")

# === Define the new policy statement ===
new_statement1 = {
    "Sid": "AllowAccountAToAccessTemplate",
    "Effect": "Allow",
    "Principal": {
        "AWS": f"arn:aws:iam::{target_account_id}:root"
    },
    "Action": "s3:*",
      "Resource": [
        f"arn:aws:s3:::{bucket_name}",
        f"arn:aws:s3:::{bucket_name}/*"
      ]
}

new_statement2 = {
      "Sid": "AllowCloudFormationAccess",
      "Effect": "Allow",
      "Principal": {
        "AWS": f"arn:aws:iam::{target_account_id}:role/{role_name}"
      },
      "Action": "s3:GetObject",
      "Resource": f"arn:aws:s3:::{bucket_name}/*"
    }


# Try to get the existing policy, or create a new one
try:
    response = s3.get_bucket_policy(Bucket=bucket_name)
    policy = json.loads(response['Policy'])
except s3.exceptions.NoSuchBucketPolicy:
    policy = {
        "Version": "2012-10-17",
        "Statement": []
    }

# Always append the new statement (no duplicate check)
policy["Statement"].append(new_statement1)
policy["Statement"].append(new_statement2)

# Update the policy
updated_policy = json.dumps(policy)
s3.put_bucket_policy(Bucket=bucket_name, Policy=updated_policy)
print("Bucket policy updated successfully.")

## Configuration

In [87]:
import json
import boto3
import logging
import csv
import io
import os
import tempfile
from typing import Any, Callable, Dict, List, Optional, Union
import sys
import botocore
import pandas as pd
from datetime import datetime
import requests

In [2]:
#start-Initial set up for the sdk env#
def default_botocore_config() -> botocore.config.Config:
    """Botocore configuration."""
    retries_config: Dict[str, Union[str, int]] = {
        "max_attempts": int(os.getenv("AWS_MAX_ATTEMPTS", "5")),
    }
    mode: Optional[str] = os.getenv("AWS_RETRY_MODE")
    if mode:
        retries_config["mode"] = mode
    return botocore.config.Config(
        retries=retries_config,
        connect_timeout=10,
        max_pool_connections=10,
        user_agent_extra=f"qs_sdk_biops",
    )

In [10]:
def _assume_role(aws_account_number, role_name, aws_region):
    sts_client = boto3.client('sts', config=default_botocore_config())
    response = sts_client.assume_role(
        RoleArn='arn:aws:iam::' + aws_account_number + ':role/' + role_name,
        RoleSessionName='quicksight'
    )
    # Storing STS credentials
    session = boto3.Session(
        aws_access_key_id=response['Credentials']['AccessKeyId'],
        aws_secret_access_key=response['Credentials']['SecretAccessKey'],
        aws_session_token=response['Credentials']['SessionToken'],
        region_name=aws_region
    )

    #logger.info("Assumed session for " + aws_account_number + " in region " + aws_region + ".")

    return session

In [69]:
def get_user_arn (session, username, region='us-east-1', namespace='default'): 
    sts_client = session.client("sts")
    account_id = sts_client.get_caller_identity()["Account"]
    if username=='root':
        arn='arn:aws:iam::'+account_id+':'+username
    else:
        arn="arn:aws:quicksight:"+region+":"+account_id+":user/"+namespace+"/"+username
    
    return arn

### Assume Role

You can assume an IAM role and create session based on the role permissions

In [ ]:
configuration = pd.read_csv("configuration.txt", sep=",")
configuration.head

In [ ]:
configuration = configuration.set_index('key').T
configuration.head

In [ ]:
# this cell is for testing purpose
value = str(configuration["source_asset"].iloc[0])
print(value)

In [64]:
#source account
sourceaccountid = str(configuration["source_account"].iloc[0])
source_role_name = str(configuration["source_role_name"].iloc[0])
source_admin_user = str(configuration["source_admin_user"].iloc[0])
aws_region='us-east-1'
sourcesession = _assume_role(sourceaccountid, source_role_name, aws_region)

In [113]:
#target account
targetaccountid = str(configuration["target_account"].iloc[0])
target_role_name = str(configuration["target_role_name"].iloc[0])
target_admin_user = str(configuration["target_admin_user"].iloc[0])
aws_region='us-east-1'
targetsession = _assume_role(targetaccountid, target_role_name, aws_region)

In [ ]:
qs_client_source = sourcesession.client('quicksight')
sts_client_source = sourcesession.client("sts")
source_account_id = sts_client_source.get_caller_identity()["Account"]
print(source_account_id)
aws_region_source = qs_client_source.meta.region_name
print(aws_region)

In [ ]:
qs_client_target = targetsession.client('quicksight')
sts_client_target = targetsession.client("sts")
target_account_id = sts_client_target.get_caller_identity()["Account"]
print(target_account_id)
aws_region_target = qs_client_target.meta.region_name
print(aws_region)

In [115]:
sourceroot=get_user_arn (sourcesession, 'root')
sourceadmin=get_user_arn (sourcesession, source_admin_user)


targetroot=get_user_arn (targetsession, 'root')
targetadmin=get_user_arn (targetsession, target_admin_user)

## Export the assets bundle from source account ##

In [ ]:
# Get current local time
current_time = datetime.now().date()

source_asset = str(configuration["source_asset"].iloc[0])
jobId = source_asset + '_' + source_account_id + '_' + str(current_time)
response = qs_client_source.start_asset_bundle_export_job(
    AwsAccountId=source_account_id,
    AssetBundleExportJobId=jobId,
    ResourceArns=[
        f"arn:aws:quicksight:us-east-1:{source_account_id}:dashboard/{source_asset}",
    ],
    IncludeAllDependencies=True,
    IncludePermissions=False,
    ExportFormat='QUICKSIGHT_JSON')


from IPython.display import JSON
JSON(response)

In [ ]:

response = qs_client_source.describe_asset_bundle_export_job(
    AwsAccountId=source_account_id,
    AssetBundleExportJobId=jobId
)



from IPython.display import JSON
JSON(response)

In [101]:
import time
def WaitForExportToComplete(jobId):
    while True:
        response = qs_client_source.describe_asset_bundle_export_job(
                    AwsAccountId=source_account_id,
                    AssetBundleExportJobId=jobId
                    )
        
        job_status = response['JobStatus']
        if job_status in ['SUCCESSFUL', 'FAILED']:
            print(f"Job finished with status: {job_status}")
            return response
        else:
            print(f"Job still running. Current status: {job_status}. Retrying in 10 seconds...")
            time.sleep(10)

In [102]:
response=WaitForExportToComplete(jobId)

Job still running. Current status: IN_PROGRESS. Retrying in 10 seconds...
Job finished with status: SUCCESSFUL


In [ ]:
# === Configuration ===
url = response['DownloadUrl']
local_path = response['ExportFormat'] + '_' + response['AssetBundleExportJobId']
local_file_name = local_path + '.zip'
bucket_name = "biops-version-control-demo-2025"
s3_key = local_path + '/' + local_file_name

# === Step 1: Download file from URL ===
response = requests.get(url)
if response.status_code == 200:
    with open(local_file_name, "wb") as f:
        f.write(response.content)
else:
    raise Exception(f"Failed to download file. Status code: {response.status_code}")

# === Step 2: Upload file to S3 ===
s3 = targetsession.client('s3')
s3.upload_file(local_file_name, bucket_name, s3_key)

print(f"File uploaded to s3://{bucket_name}/{s3_key}")



In [ ]:
s3path = f"{bucket_name}/{s3_key}"
s3url = f"https://{bucket_name}.s3.us-east-1.amazonaws.com/{s3_key}"
s3uri = f"s3://{bucket_name}/{s3_key}"
print(s3uri)

## Import the assets bundle into target account ##

In [ ]:
importjobid = source_asset + '_' + target_account_id + '_' + str(current_time)
response = qs_client_target.start_asset_bundle_import_job(
    AwsAccountId=target_account_id,
    AssetBundleImportJobId=importjobid,
    AssetBundleImportSource={
        'S3Uri': s3uri
    },
    FailureAction='ROLLBACK'
)

from IPython.display import JSON
JSON(response)

In [ ]:
response = qs_client_target.describe_asset_bundle_import_job(
    AwsAccountId=target_account_id,
    AssetBundleImportJobId=importjobid
)



from IPython.display import JSON
JSON(response)

In [107]:
import time
def WaitForImportToComplete(jobId):
    while True:
        response = qs_client_target.describe_asset_bundle_import_job(
                    AwsAccountId=target_account_id,
                    AssetBundleImportJobId=importjobid
                    )
        
        job_status = response['JobStatus']
        if job_status in ['SUCCESSFUL', 'FAILED', 'FAILED_ROLLBACK_COMPLETED']:
            print(f"Job finished with status: {job_status}")
            return response
        else:
            print(f"Job still running. Current status: {job_status}. Retrying in 10 seconds...")
            time.sleep(10)

In [108]:
response=WaitForImportToComplete(importjobid)

Job still running. Current status: IN_PROGRESS. Retrying in 10 seconds...
Job finished with status: SUCCESSFUL


In [ ]:
response = qs_client_target.search_dashboards(
    AwsAccountId=target_account_id,
    Filters=[
        {
            'Operator': 'StringLike',
            'Name': 'DASHBOARD_NAME',
            'Value': 'BIOpsDemo'
        },
    ]
)

from IPython.display import JSON
JSON(response)

In [ ]:
print(targetadmin)
print(response['DashboardSummaryList'][0]['DashboardId'])
target_asset = response['DashboardSummaryList'][0]['DashboardId']

In [ ]:

response = qs_client_target.update_dashboard_permissions(
    AwsAccountId=target_account_id,
    DashboardId=target_asset,
    GrantPermissions=[
        {
            'Principal': targetadmin,
            'Actions': [
                "quicksight:DescribeDashboard",
              "quicksight:ListDashboardVersions",
              "quicksight:UpdateDashboardPermissions",
              "quicksight:QueryDashboard",
              "quicksight:UpdateDashboard",
              "quicksight:DeleteDashboard",
              "quicksight:DescribeDashboardPermissions",
              "quicksight:UpdateDashboardPublishedVersion"
            ]
        }
    ]
    
)

from IPython.display import JSON
JSON(response)

In [ ]:
response = qs_client_target.describe_dashboard(
    AwsAccountId=target_account_id,
    DashboardId=target_asset
)

from IPython.display import JSON
JSON(response)

In [ ]:
from IPython.display import JSON

datasets = response['Dashboard']['Version']['DataSetArns']
for dataset in datasets:
    dsid = dataset.split(":")[-1].split("/")[-1]
    res = qs_client_target.update_data_set_permissions(
    AwsAccountId=target_account_id,
    DataSetId = dsid,
    GrantPermissions=[
        {
            'Principal': targetadmin,
            'Actions': [
                 "quicksight:UpdateDataSetPermissions",
              "quicksight:DescribeDataSet",
              "quicksight:DescribeDataSetPermissions",
              "quicksight:PassDataSet",
              "quicksight:DescribeIngestion",
              "quicksight:ListIngestions",
              "quicksight:UpdateDataSet",
              "quicksight:DeleteDataSet",
              "quicksight:CreateIngestion",
              "quicksight:CancelIngestion"
            ]
        }
    ]
    
  )
    JSON(res)
    print(dsid + " permissions are updated" )
    
  
    